# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [3]:
# Install openpyxl and CCXT
!pip install openpyxl ccxt

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path(".")
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)
    return filename


In [5]:
# scrape_candles_to_csv('btc_usdt_15m_e_20230501.csv', 'binance', 5, 'BTC/USDT', '15m', '2015-05-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

In [6]:
from datetime import datetime, timedelta

def get_date_one_month_ago():
    # Получаем текущую дату
    today = datetime.now()

    # Вычисляем дату на месяц назад
    one_month_ago = today - timedelta(days=365)

    # Форматируем дату в нужный формат
    formatted_date = one_month_ago.strftime("%Y-%m-%dT%H:%M:%SZ")

    return formatted_date

# Пример использования функции
date_one_month_ago = get_date_one_month_ago()
print(date_one_month_ago)


exchange = 'binance'
ticker = 'BTC/USDT'
timeframe = '1m'
file = scrape_candles_to_csv(
    filename=f'{date_one_month_ago.replace(":", "").replace("/", "")}_{timeframe}_{ticker.replace("/", "")}.csv',
    exchange_id=exchange,
    max_retries=10,
    symbol=ticker,
    timeframe=timeframe,
    since=date_one_month_ago,
    limit=1000
)

print(file)


2022-10-13T23:53:17Z


Fetched 1000 BTC/USDT candles from 2023-10-13T04:14:00.000Z to 2023-10-13T20:53:00.000Z
1000 BTC/USDT candles in total from 2023-10-13T04:14:00.000Z to 2023-10-13T20:53:00.000Z
Fetched 1000 BTC/USDT candles from 2023-10-12T11:34:00.000Z to 2023-10-13T04:13:00.000Z
2000 BTC/USDT candles in total from 2023-10-12T11:34:00.000Z to 2023-10-13T20:53:00.000Z
Fetched 1000 BTC/USDT candles from 2023-10-11T18:54:00.000Z to 2023-10-12T11:33:00.000Z
3000 BTC/USDT candles in total from 2023-10-11T18:54:00.000Z to 2023-10-13T20:53:00.000Z
Fetched 1000 BTC/USDT candles from 2023-10-11T02:14:00.000Z to 2023-10-11T18:53:00.000Z
4000 BTC/USDT candles in total from 2023-10-11T02:14:00.000Z to 2023-10-13T20:53:00.000Z
Fetched 1000 BTC/USDT candles from 2023-10-10T09:34:00.000Z to 2023-10-11T02:13:00.000Z
5000 BTC/USDT candles in total from 2023-10-10T09:34:00.000Z to 2023-10-13T20:53:00.000Z
Fetched 1000 BTC/USDT candles from 2023-10-09T16:54:00.000Z to 2023-10-10T09:33:00.000Z
6000 BTC/USDT candles in to

In [7]:
import pandas as pd

data = pd.read_csv('2022-10-13T235317Z_1m_BTCUSDT.csv')

In [9]:
import pandas as pd

# Загрузите CSV-файл в DataFrame
df = pd.read_csv('2022-10-13T235317Z_1m_BTCUSDT.csv', header=None)

# Переименуйте столбцы
df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']

# Удалите столбец "Timestamp"
df = df.drop('Timestamp', axis=1)

# Сохраните DataFrame в новый CSV-файл
df.to_csv('aggregated_dataset.csv', index=False, header=['Open', 'High', 'Low', 'Close', 'Volume'])
